In [93]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

import warnings
warnings.filterwarnings("ignore")

In [94]:
# read dataset
# https://archive.ics.uci.edu/dataset/352/online+retail
df=pd.read_excel("Online Retail.xlsx")

In [95]:
#check dimensions of data
df.shape

(541909, 8)

In [96]:
df.sample(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
145182,548870,21623,VINTAGE UNION JACK MEMOBOARD,1,2011-04-04 14:41:00,9.95,14606.0,United Kingdom
250768,559048,21975,PACK OF 60 DINOSAUR CAKE CASES,720,2011-07-05 15:48:00,0.40,13027.0,United Kingdom
528607,580729,23395,BELLE JARDINIERE CUSHION COVER,2,2011-12-05 17:24:00,10.79,NaN,United Kingdom
284042,561820,23201,JUMBO BAG ALPHABET,2,2011-07-29 16:00:00,4.13,NaN,United Kingdom
174832,551855,21912,VINTAGE SNAKES & LADDERS,3,2011-05-04 15:08:00,3.75,12836.0,United Kingdom
331472,566017,22386,JUMBO BAG PINK POLKADOT,10,2011-09-08 12:27:00,2.08,13330.0,United Kingdom
81054,543107,23230,WRAP ALPHABET DESIGN,25,2011-02-03 12:27:00,0.42,13800.0,United Kingdom
281301,561515,84031B,CHARLIE LOLA BLUE HOT WATER BOTTLE,1,2011-07-27 15:16:00,4.13,NaN,United Kingdom
185755,552825,22355,CHARLOTTE BAG SUKI DESIGN,10,2011-05-11 13:18:00,0.85,15296.0,United Kingdom
535540,581217,22090,PAPER BUNTING RETROSPOT,1,2011-12-08 09:20:00,5.79,NaN,United Kingdom


In [97]:
df.dtypes

InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID            float64
Country                object
dtype: object

In [98]:
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [99]:
# Droping description with null
df.dropna(axis=0,subset=["Description"],inplace=True)

In [100]:
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description         0
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     133626
Country             0
dtype: int64

In [101]:
# remove extra white spaces in description
df['Description'] = df['Description'].str.replace(r'\s+', ' ', regex=True)
df['Description'] = df['Description'].str.strip()

In [102]:
# checking for cancelld orders
df['InvoiceNo'].str.contains('C').sum()

9288

In [103]:
# Remove cancelled orders
df = df[~df['InvoiceNo'].str.contains('C', na=False)]

In [104]:
df['InvoiceNo'].str.contains('C').sum()

0

In [105]:
#convert Invoice no to string
df['InvoiceNo'] = df['InvoiceNo'].astype(str)

In [106]:
print("total number of countries",df["Country"].nunique())
df["Country"].value_counts(normalize=True)

total number of countries 38


United Kingdom          0.915283
Germany                 0.017023
France                  0.015829
EIRE                    0.014862
Spain                   0.004678
Netherlands             0.004449
Belgium                 0.003824
Switzerland             0.003703
Portugal                0.002826
Australia               0.002231
Norway                  0.002018
Italy                   0.001427
Channel Islands         0.001408
Finland                 0.001290
Cyprus                  0.001156
Sweden                  0.000849
Unspecified             0.000840
Austria                 0.000749
Denmark                 0.000715
Poland                  0.000621
Japan                   0.000604
Israel                  0.000555
Hong Kong               0.000535
Singapore               0.000418
Iceland                 0.000343
USA                     0.000337
Canada                  0.000284
Greece                  0.000273
Malta                   0.000211
United Arab Emirates    0.000128
European C

In [107]:
# Lets focus on United Kingdom
grouped_df=(df[df["Country"]=="Germany"]
           .groupby(["InvoiceNo","Description"])
           .size() # counts
           .unstack() 
           .fillna(0)
           .reset_index()
           .set_index("InvoiceNo"))

In [108]:
grouped_df.head()

Description,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 IVORY ROSE PEG PLACE SETTINGS,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE SKULLS,...,YULETIDE IMAGES GIFT WRAP SET,ZINC BOX SIGN HOME,ZINC FOLKART SLEIGH BELLS,ZINC HEART LATTICE T-LIGHT HOLDER,ZINC HEART T-LIGHT HOLDER,ZINC METAL HEART DECORATION,ZINC STAR T-LIGHT HOLDER,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC WILLIE WINKIE CANDLE STICK
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536527,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536840,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536861,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536967,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536983,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [109]:
# all positive values are converted to 1 and anything less than 0 to 0
def custom_encoding(x):
    if x<=0:
        return 0
    if x>=1:
        return 1
grouped_df=grouped_df.applymap(custom_encoding)
grouped_df.drop("POSTAGE",inplace=True,axis=1)

In [110]:
grouped_df.sample(10)

Description,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 IVORY ROSE PEG PLACE SETTINGS,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE SKULLS,...,YULETIDE IMAGES GIFT WRAP SET,ZINC BOX SIGN HOME,ZINC FOLKART SLEIGH BELLS,ZINC HEART LATTICE T-LIGHT HOLDER,ZINC HEART T-LIGHT HOLDER,ZINC METAL HEART DECORATION,ZINC STAR T-LIGHT HOLDER,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC WILLIE WINKIE CANDLE STICK
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
552878,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
541093,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
552008,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
543121,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
545988,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
555162,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
550354,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
539446,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
575750,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [119]:
frequent_itemset=apriori(grouped_df,min_support=0.05,use_colnames=True)

In [120]:
rules=association_rules(frequent_itemset,metric="confidence",min_threshold=0.3)

In [121]:
 rules.sort_values(by="lift",ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
9,(WOODLAND CHARLOTTE BAG),(RED RETROSPOT CHARLOTTE BAG),0.126915,0.070022,0.059081,0.465517,6.648168,0.050194,1.739959,0.973081
10,(RED RETROSPOT CHARLOTTE BAG),(WOODLAND CHARLOTTE BAG),0.070022,0.126915,0.059081,0.843750,6.648168,0.050194,5.587746,0.913551
0,(PLASTERS IN TIN WOODLAND ANIMALS),(PLASTERS IN TIN CIRCUS PARADE),0.137856,0.115974,0.067834,0.492063,4.242887,0.051846,1.740427,0.886524
1,(PLASTERS IN TIN CIRCUS PARADE),(PLASTERS IN TIN WOODLAND ANIMALS),0.115974,0.137856,0.067834,0.584906,4.242887,0.051846,2.076984,0.864580
5,(PLASTERS IN TIN WOODLAND ANIMALS),(PLASTERS IN TIN SPACEBOY),0.137856,0.107221,0.061269,0.444444,4.145125,0.046488,1.607002,0.880076
6,(PLASTERS IN TIN SPACEBOY),(PLASTERS IN TIN WOODLAND ANIMALS),0.107221,0.137856,0.061269,0.571429,4.145125,0.046488,2.011670,0.849877
11,(ROUND SNACK BOXES SET OF 4 FRUITS),(ROUND SNACK BOXES SET OF4 WOODLAND),0.157549,0.245077,0.131291,0.833333,3.400298,0.092679,4.529540,0.837922
12,(ROUND SNACK BOXES SET OF4 WOODLAND),(ROUND SNACK BOXES SET OF 4 FRUITS),0.245077,0.157549,0.131291,0.535714,3.400298,0.092679,1.814509,0.935072
13,(SPACEBOY LUNCH BOX),(ROUND SNACK BOXES SET OF4 WOODLAND),0.102845,0.245077,0.070022,0.680851,2.778116,0.044817,2.365427,0.713415
2,(ROUND SNACK BOXES SET OF 4 FRUITS),(PLASTERS IN TIN CIRCUS PARADE),0.157549,0.115974,0.050328,0.319444,2.754455,0.032057,1.298977,0.756070
